In [1]:
from __future__ import print_function
from __future__ import division
import json
import py_crepe
import datetime
import numpy as np
import data_helpers
import data
import string
import pandas as pd
import autoEncoder
np.random.seed(0123)  # for reproducibility


Using Theano backend.


In [2]:
# set parameters:

subset = None

#Whether to save model parameters
save = False
model_name_path = 'params/crepe_model.json'
model_weights_path = 'params/auto_model_weights.h5'

#Maximum length. Longer gets chopped. Shorter gets padded.
maxlen = 1014

#Model params
#Filters for conv layers
nb_filter = 128 #initially 256
#Number of units in the dense layer
dense_outputs = 512 #Initially 1024
#Conv layer kernel size
filter_kernels = [7, 7, 3, 3, 3, 3]
#Number of units in the final output layer. Number of classes.

#Compile/fit params
batch_size = 32
nb_epoch = 10

In [3]:
print('Loading data...')
#Expect x to be a list of sentences. Y to be a one-hot encoding of the
#categories.

### 515-1122-122 and 1573 with remove 6 layers
#authorlist=[121, 479 , 649 ]
#doc_id = 14706

authorlist=[3816, 1416, 1472]
doc_id = [3756, 3253, 3399, 3393, 13442, 14907]
cat_output = len(authorlist) #binary in the last layer

# def main(authorlist, doc_id):
    
    
((trainX, trainY), (valX, valY)) = data_helpers.load_ag_data(authors = authorlist, docID = doc_id)

print('Creating vocab...')
vocab, reverse_vocab, vocab_size, check = data_helpers.create_vocab_set()


#trainX = data_helpers.encode_data(trainX, maxlen, vocab, vocab_size, check)
#test_data = data_helpers.encode_data(valX, maxlen, vocab, vocab_size, check)

print('Build model...')

classes = len(authorlist)

#(model, model_weights_path) = autoEncoder.auto_model(maxlen, vocab_size)
(model, model_weights_path) = autoEncoder.fcc_auto_model(maxlen, vocab_size,
                                                         model_weights_path,
                                                         dense_outputs, classes)


Loading data...
SELECT author_id, doc_content FROM aman_content WHERE author_id IN (3816, 1416, 1472) AND doc_id NOT IN (3756, 3253, 3399, 3393, 13442, 14907) ;
Execution completed
Read completed
Number of rows: 501
author_id       int64
doc_content    object
dtype: object
Data Frame created: Shape: (98052, 2)
Author:  1416  Size: 33336
Author:  1472  Size:  4936
Author:  3816  Size: 59780
Min: 4936
Max: 59780
Authors [1416, 1472, 3816].
Found 14808 texts.
Found 14808 labels.
Creating vocab...
Build model...


In [4]:
vocab_size

101

In [5]:
print('Fit model...')
initial = datetime.datetime.now()
for e in xrange(nb_epoch):
    xi, yi = data_helpers.shuffle_matrix(trainX, trainY)
    xi_test, yi_test = data_helpers.shuffle_matrix(valX, valY)
    if subset:
        batches = data_helpers.mini_batch_generator(xi[:subset], yi[:subset],
                                                    vocab, vocab_size, check,
                                                    maxlen,
                                                    batch_size=batch_size)
    else:
        batches = data_helpers.mini_batch_generator(xi, yi, vocab, vocab_size,
                                                    check, maxlen,
                                                    batch_size=batch_size)

    test_batches = data_helpers.mini_batch_generator(xi_test, yi_test, vocab,
                                                     vocab_size, check, maxlen,
                                                     batch_size=batch_size)

    accuracy = 0.0
    loss = 0.0
    step = 1
    start = datetime.datetime.now()
    print('Epoch: {}'.format(e))
    for x_train, y_train in batches:
        
        f = model.train_on_batch(x_train, y_train)
        loss += f[0]
        loss_avg = loss / step
        accuracy += f[1]
        accuracy_avg = accuracy / step
        if step % 100 == 0:
            print('  Step: {}'.format(step))
            print('\tLoss: {}. Accuracy: {}'.format(loss_avg, accuracy_avg))
        step += 1

    test_accuracy = 0.0
    test_loss = 0.0
    test_step = 1
    
    for x_test_batch, y_test_batch in test_batches:
        f_ev = model.test_on_batch(x_test_batch, y_test_batch)
        test_loss += f_ev[0]
        test_loss_avg = test_loss / test_step
        test_accuracy += f_ev[1]
        test_accuracy_avg = test_accuracy / test_step
        test_step += 1
    stop = datetime.datetime.now()
    e_elap = stop - start
    t_elap = stop - initial
    print('Epoch {}. Loss: {}. Accuracy: {}\nEpoch time: {}. Total time: {}\n'.format(e, test_loss_avg, test_accuracy_avg, e_elap, t_elap))

if save:
    print('Saving model params...')
    json_string = model.to_json()
    with open(model_name_path, 'w') as f:
        json.dump(json_string, f)

model.save_weights(model_weights_path)

'''
import cPickle as pickle
with open('sgd.pickle', 'wb') as handle:
    pickle.dump(sgd, handle, protocol=pickle.HIGHEST_PROTOCOL)
'''


Fit model...
Epoch: 0
  Step: 100
	Loss: 1.18011718512. Accuracy: 0.3278125
  Step: 200
	Loss: 1.13938054204. Accuracy: 0.32953125
  Step: 300
	Loss: 1.12579505285. Accuracy: 0.328541666667
Epoch 0. Loss: 1.09871772797. Accuracy: 0.318436380035
Epoch time: 0:01:05.422985. Total time: 0:01:05.727957

Epoch: 1
  Step: 100
	Loss: 1.09866985798. Accuracy: 0.3296875
  Step: 200
	Loss: 1.09866061866. Accuracy: 0.32765625
  Step: 300
	Loss: 1.09863682389. Accuracy: 0.3309375
Epoch 1. Loss: 1.09880826678. Accuracy: 0.318959080083
Epoch time: 0:01:03.069619. Total time: 0:02:09.105622

Epoch: 2
  Step: 100
	Loss: 1.09862119555. Accuracy: 0.3365625
  Step: 200
	Loss: 1.09865185857. Accuracy: 0.33359375
  Step: 300
	Loss: 1.09862994313. Accuracy: 0.335729166667
Epoch 2. Loss: 1.09887279105. Accuracy: 0.317913679827
Epoch time: 0:01:02.286320. Total time: 0:03:11.693442

Epoch: 3
  Step: 100
	Loss: 1.09878999114. Accuracy: 0.3215625
  Step: 200
	Loss: 1.09871055543. Accuracy: 0.329375
  Step: 300


"\nimport cPickle as pickle\nwith open('sgd.pickle', 'wb') as handle:\n    pickle.dump(sgd, handle, protocol=pickle.HIGHEST_PROTOCOL)\n"

In [6]:
del trainX, trainY, valX, valY

In [7]:
model.load_weights(model_weights_path)

from keras.optimizers import SGD
sgd = SGD(lr=0.01, momentum=0.9, nesterov= True)

# Compile model again (required to make predictions)
model.compile(loss='categorical_crossentropy', optimizer=sgd,
              metrics=['accuracy'])



In [8]:
test_binary = []
for docs in doc_id:
    (testX, testY) = data_helpers.load_doc_data(authors = authorlist, docID = docs)
    testX = data_helpers.encode_data(testX, maxlen, vocab, vocab_size, check)
    predY = np.array(model.predict(testX, batch_size=batch_size))
    testY = np.array(testY)
    predY = predY.mean(axis = 0)
    testY = testY.mean(axis = 0)
    predLocation = predY.tolist().index(max(predY))
    if predLocation == testY:
        test_binary.append(1)
    else:
        test_binary.append(0)
    
    
    from IPython.display import clear_output
    clear_output()
        



In [9]:
#predY = np.array(model.predict(testX, batch_size=batch_size))

In [10]:
#predY


In [11]:
#predY.mean(axis = 0)

In [12]:
test_binary

[0, 0, 0, 0, 1, 1]

In [13]:
"""
feature_model = py_crepe.build_feature_model()
feature_trainX = feature_model.predict(trainX)
feature_testX = feature_model.predict(testX)
"""

'\nfeature_model = py_crepe.build_feature_model()\nfeature_trainX = feature_model.predict(trainX)\nfeature_testX = feature_model.predict(testX)\n'